In [1]:
import random
import re
import pickle

import openai

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.l2 import Ether
from scapy.layers.inet import IP, TCP, ICMP

In [14]:
packets_per_request = 5

In [3]:
with open("../data/text/three_summaries.txt","r") as f:
    packets_summary = f.read().splitlines()

In [4]:
len(packets_summary)

9999

In [5]:
engines = openai.Engine.list()


In [17]:
responses = pickle.load( open("../data/pickle/davinci_pairs2.pkl", "rb" ))


In [104]:
len(responses)

199

In [105]:
for i in range(1):
    summaries = random.choices(packets_summary,k=packets_per_request)
    query = "This is a packet summary:\n"
    query += "192.168.1.194 → 192.168.1.190 ICMP 100 Echo (ping) request id=0x9015, seq=18917\n"
    query += "This is the python code to generate the packet:\n"
    query += 'scapy.IP(src="192.168.1.194", dst="192.168.1.190")/scapy.ICMP(type=8, id=0x9015, seq=18917)\n\n'
    query += "This is a packet summary:\n"
    query += "192.168.1.190 → 192.168.1.152 DNS 193 Standard query response 0x0971 No such name AAAA a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com SOA ns-1926.awsdns-48.co.uk\n"
    query += "This is the python code to generate the packet:\n"
    query += 'IP(src="192.168.1.190", dst="192.168.1.152")/DNS(id=0x0971, qr=1, opcode=0, rcode=3, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com", qtype="SOA", qclass="IN"))\n\n'

    query += "These are the packet summaries:\n"
    text_sum = ""
    for summary in summaries:
        query += summary + "\n"
        text_sum += summary + "\n"

    query += "\nGenerate python code for creating these packets with scapy framework and put them all in a list named pkt_list."

    #print(query)

    completion = openai.Completion.create(engine="text-davinci-003", prompt=query,max_tokens=2600,temperature=0.1)
    completion["prompt_summary"] = text_sum
    responses.append(completion)

    #print(responses[-1].choices[0].text)
    
    exec(responses[-1].choices[0].text)
    pickle.dump(responses, open( "../data/pickle/davinci_pairs2.pkl", "wb" ) )
    print(len(responses))


200


In [42]:
exec(responses[-1].choices[0].text)
print(pkt_list)

[<IP  frag=0 proto=icmp src=192.168.1.190 dst=192.168.1.194 |<ICMP  type=echo-reply id=0x9015 seq=0x4b4a |>>, <IP  frag=0 proto=icmp src=192.168.1.190 dst=192.168.1.193 |<ICMP  type=echo-reply id=0x1 seq=0xffb |>>, <IP  src=192.168.1.152 dst=192.168.1.190 |<DNS  id=37414 qr=0 opcode=QUERY rcode=ok qdcount=1 ancount=0 nscount=0 arcount=0 qd=<DNSQR  qname='a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com.' qtype=AAAA qclass=IN |> |>>, <IP  frag=0 proto=icmp src=192.168.1.193 dst=192.168.1.190 |<ICMP  type=echo-request id=0x1 seq=0x1453 |>>, <IP  frag=0 proto=icmp src=192.168.1.190 dst=192.168.1.192 |<ICMP  type=echo-reply id=0x112d seq=0x1658 |>>]


In [85]:
print(responses[-1].prompt_summary)
print(responses[-1].choices[0].text)

192.168.1.193 → 192.168.1.190 ICMP 76 Echo (ping) request id=0x0001, seq=5129
192.168.1.192 → 192.168.1.190 ICMP 100 Echo (ping) request id=0x112d, seq=6406
2405:6e00:10ce:2c00:20c:29ff:feee:e07a → 2600:9000:5301:200::1 DNS 120 Standard query 0x30e9 A shavar.services.mozilla.com OPT
2001:503:ba3e::2:30 → 2405:6e00:10ce:2c00:20c:29ff:feee:e07a DNS 1161 Standard query response 0x4aa2 NS <Root> NS e.root-servers.net NS h.root-servers.net NS l.root-servers.net NS i.root-servers.net NS a.root-servers.net NS d.root-servers.net NS c.root-servers.net NS b.root-servers.net NS j.root-servers.net NS k.root-servers.net NS g.root-servers.net NS m.root-servers.net NS f.root-servers.net RRSIG A 192.203.230.10 AAAA 2001:500:a8::e A 198.97.190.53 AAAA 2001:500:1::53 A 199.7.83.42 AAAA 2001:500:9f::42 A 192.36.148.17 AAAA 2001:7fe::53 A 198.41.0.4 AAAA 2001:503:ba3e::2:30 A 199.7.91.13 AAAA 2001:500:2d::d A 192.33.4.12 AAAA 2001:500:2::c A 199.9.14.201 AAAA 2001:500:200::b A 192.58.128.30 AAAA 2001:503:

In [44]:
pickle.dump(responses, open( "../data/pickle/davinci_pairs2.pkl", "wb" ) )

In [79]:
fixed_text = """

pkt_list = [scapy.IP(src="192.168.1.193", dst="192.168.1.190")/scapy.ICMP(type=8, id=0x0001, seq=5487),
            scapy.IP(src="192.168.1.194", dst="192.168.1.190")/scapy.ICMP(type=8, id=0x9015, seq=18999),
            scapy.IP(src="192.168.1.194", dst="192.168.1.190")/scapy.ICMP(type=8, id=0x9015, seq=18871),
            scapy.IP(src="192.168.1.190", dst="192.168.1.194")/scapy.ICMP(type=0, id=0x9015, seq=18605),
            scapy.IP(src="192.168.1.190", dst="192.5.6.30")/scapy.DNS(id=0x03bc, qr=0, opcode=0, qdcount=1, qd=DNSQR(qname="phicdn.net", qtype="DS", qclass="IN"))]
"""



In [80]:
exec(fixed_text)

In [81]:
pkt_list

[<IP  frag=0 proto=icmp src=192.168.1.193 dst=192.168.1.190 |<ICMP  type=echo-request id=0x1 seq=0x156f |>>,
 <IP  frag=0 proto=icmp src=192.168.1.194 dst=192.168.1.190 |<ICMP  type=echo-request id=0x9015 seq=0x4a37 |>>,
 <IP  frag=0 proto=icmp src=192.168.1.194 dst=192.168.1.190 |<ICMP  type=echo-request id=0x9015 seq=0x49b7 |>>,
 <IP  frag=0 proto=icmp src=192.168.1.190 dst=192.168.1.194 |<ICMP  type=echo-reply id=0x9015 seq=0x48ad |>>,
 <IP  src=192.168.1.190 dst=192.5.6.30 |<DNS  id=956 qr=0 opcode=QUERY qdcount=1 qd=<DNSQR  qname='phicdn.net.' qtype=DS qclass=IN |> |>>]

In [82]:
responses[-1].choices[0].text = fixed_text

In [60]:
print(responses[-1].prompt_summary)
print(responses[-1].choices[0].text)

192.168.1.190 → 192.168.1.192 DNS 222 Standard query response 0x1565 A daisy.ubuntu.com A 162.213.33.132 A 162.213.33.108 NS ns1.canonical.com NS ns3.canonical.com NS ns2.canonical.com A 91.189.94.173 A 91.189.95.3 A 91.189.91.139
192.168.1.190 → 192.168.1.192 ICMP 100 Echo (ping) reply id=0x112d, seq=5807
192.168.1.190 → 192.168.1.192 ICMP 100 Echo (ping) reply id=0x112d, seq=6340
192.168.1.190 → 192.168.1.192 ICMP 100 Echo (ping) reply id=0x112d, seq=6600
192.168.1.194 → 192.168.1.190 ICMP 100 Echo (ping) request id=0x9015, seq=19885



pkt_list = [scapy.IP(src="192.168.1.190", dst="192.168.1.192")/DNS(id=0x1565, qr=1, opcode=0, rcode=0, qdcount=1, ancount=5, nscount=3, arcount=0, qd=DNSQR(qname="daisy.ubuntu.com", qtype="A", qclass="IN"), an=DNSRR(rrname="daisy.ubuntu.com", type="A", rclass="IN", ttl=172800, rdata="162.213.33.132")/DNSRR(rrname="daisy.ubuntu.com", type="A", rclass="IN", ttl=172800, rdata="162.213.33.108")/DNSRR(rrname="daisy.ubuntu.com", type="NS", rclass="IN", ttl=

In [103]:
del responses[-1]

In [22]:

pkt_list = []

pkt_list.append(scapy.IP(src="192.168.1.190", dst="192.168.1.193")/scapy.ICMP(type=76, id=0x0001, seq=4091))

pkt_list.append(scapy.IP(src="192.168.1.194", dst="192.168.1.190")/scapy.ICMP(type=100, id=0x9015, seq=18476))

pkt_list.append(scapy.IP(src="192.168.1.192", dst="192.168.1.190")/scapy.ICMP(type=100, id=0x112d, seq=5799))

pkt_list.append(scapy.IP(src="192.168.1.194", dst="192.168.1.190")/scapy.ICMP(type=100, id=0x9015, seq=19355))

pkt_list.append(scapy.IP(src="192.168.1.193", dst="192.168.1.190")/scapy.ICMP(type=76, id=0x0001, seq=4322))

pkt_list.append(scapy.IP(src="192.168.1.192", dst="192.168.1.190")/scapy.ICMP(type=100, id=0x112d, seq=5804))

pkt_list.append(scapy.IP(src="192.168.1.190", dst="192.168.1.194")/scapy.ICMP(type=100, id=0x9015, seq=19228))

pkt_list.append(scapy.IP(src="192.168.1.190", dst="192.168.1.194")/scapy.ICMP(type=100, id=0x9015, seq=19718))

pkt_list.append(scapy.IP(src="192.168.1.193", dst="192.168.1.190")/scapy.ICMP(type=76, id=0x0001, seq=5336))

In [24]:
with open("../data/pcapgenerated_curie_commands.pcap", "wb") as f:
    wrpcap(f, pkt_list)

In [32]:
print(responses[0])

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\npkt_list = [scapy.IP(src=\"192.168.1.192\", dst=\"192.168.1.190\")/scapy.ICMP(type=8, id=0x112d, seq=5619),\n            scapy.IP(src=\"192.168.1.194\", dst=\"192.168.1.190\")/scapy.ICMP(type=8, id=0x9015, seq=19673),\n            scapy.IP(src=\"192.168.1.194\", dst=\"192.168.1.190\")/scapy.ICMP(type=8, id=0x9015, seq=18836),\n            scapy.IP(src=\"192.168.1.190\", dst=\"192.168.1.193\")/scapy.ICMP(type=0, id=0x0001, seq=4886),\n            scapy.IP(src=\"192.168.1.190\", dst=\"192.168.1.193\")/scapy.ICMP(type=0, id=0x0001, seq=4305)]"
    }
  ],
  "created": 1677576757,
  "id": "cmpl-6oqdZkXJZj4oXycbtlpbzYMv4Jzfz",
  "model": "text-davinci-003",
  "object": "text_completion",
  "prompt_summary": "192.168.1.192 \u2192 192.168.1.190 ICMP 100 Echo (ping) request id=0x112d, seq=5619\n192.168.1.194 \u2192 192.168.1.190 ICMP 100 Echo (ping) request id=0x9015, seq=19673\n19